In [1]:
import numpy as np

In [2]:
# softmax forward, backward
class Softmax:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None

    def forward(self, x):
        self.out = Softmax.softmax(x)
        return self.out

    def backward(self, dout):
        dx = self.out * dout
        sumdx = np.sum(dx, axis=1, keepdims=True)
        dx -= self.out * sumdx
        return dx
    
    # softmax 연산
    @staticmethod
    def softmax(x):
        if x.ndim == 2:
            x = x - x.max(axis=1, keepdims=True)
            x = np.exp(x)
            x /= x.sum(axis=1, keepdims=True)
        elif x.ndim == 1:
            x = x - np.max(x)
            x = np.exp(x) / np.sum(np.exp(x))
    
        return x

In [3]:
# 디코더의 현재 히든 상태값과 인코더의 각 상태별 히든값의 내적 및 정규화
class AttentionWeight:
    def __init__(self):
        self.params, self.grads = [], []
        self.softmax = Softmax()
        self.cache = None

    
    def forward(self, hs, h):
        """
        인코더의 각 상태값과 디코더의 현재 히든값을 내적 및 정규화
        결국 현재 디코더의 히든 스테이트 기준, 인코더의 모든 단어들의 히든 스테이트의 유사도 측정
        :return: 
        :param hs: 인코더의 전체 히든 스테이트
        :param h: 디코더의 현재 히든 스테이트
        :return: 내적하고 정규화한거
        """
        num_sample, time_stap, hidden_size = hs.shape

        # h의 원래 shape은 (num_sample, hidden_size)인데 hs랑 내적해야해서 같은 형태로 만들어줌
        hr = h.reshape(num_sample, 1, hidden_size)#.repeat(T, axis=1)
        t = hs * hr  # 같은 shape의 numpy array를 곱하면 원소별 곱
        s = np.sum(t, axis=2)  # 원소별 덧셈 = 내적한 결과
        a = self.softmax.forward(s)  # 정규화를 위한 softmax

        self.cache = (hs, hr)  # 역전파를 위해 저장☆
        return a

    def backward(self, da):
        """
        상위로부터 받은 da로 역전파
        sofrmax -> sum -> 원소곱 순으로 역전파 진행
        :param da: 상위 계층으로부터 전달받은 a의 오차값
        :return: hs와 h의 오차
        """
        hs, hr = self.cache
        num_sample, time_stap, hidden_size = hs.shape

        ds = self.softmax.backward(da)  # softmax 역전파하면 ds
        dt = ds.reshape(num_sample, time_stap, 1).repeat(hidden_size, axis=2)  # 덧셈노드의 역전파는 repeat노드~
        # t가 hs * hr이니까 각각 반대로 곱해주자 -> 곱셈노드는 반대로 곱해주기~
        dhs = dt * hr
        dhr = dt * hs
        dh = np.sum(dhr, axis=1)  # repeat노드였으니까 sum노드~

        return dhs, dh
   

## AttentionWeight 테스트
 

In [4]:
num_sample = 10
time_stap = 5
hidden_size = 4

hs = np.random.randn(num_sample, time_stap, hidden_size)
hs

h = np.random.randn(num_sample, hidden_size)
h

hr = h.reshape(num_sample, 1, hidden_size).repeat(time_stap, axis=1)
hr

t = hs * hr
t

s = np.sum(t, axis=2)
s

softmax = Softmax()
a = softmax.forward(s)
a

array([[0.11195273, 0.16978945, 0.09138643, 0.36925081, 0.2576206 ],
       [0.20106812, 0.1687147 , 0.1702468 , 0.14653598, 0.3134344 ],
       [0.27515615, 0.21627004, 0.39823815, 0.01212188, 0.09821378],
       [0.11936457, 0.0697606 , 0.07891963, 0.41000414, 0.32195106],
       [0.01158808, 0.56577483, 0.29054582, 0.06214907, 0.0699422 ],
       [0.35247731, 0.05567077, 0.2022596 , 0.27362803, 0.11596429],
       [0.00071705, 0.47902672, 0.44651269, 0.07109236, 0.00265119],
       [0.05590267, 0.18602931, 0.47905771, 0.01412347, 0.26488684],
       [0.27253952, 0.29879283, 0.07418869, 0.16821585, 0.18626311],
       [0.09951457, 0.38910379, 0.0379079 , 0.01428984, 0.4591839 ]])

In [5]:
class WeightSum:
    def __init__(self):
        self.params, self.grads = [], []
        self.cache = None

    def forward(self, hs, a):
        """
        인코더의 모든 히든 스테이트 hs와 각 단어들의 가중합
        :param hs: 인코더의 모든 히든 스테이트
        :param a: 각 단어별 가중치
        :return: 가중합된 맥락벡터
        """
        num_sample, time_stap, hidden_size = hs.shape

        # 위랑 거의 같음
        ar = a.reshape(num_sample, time_stap, 1)#.repeat(T, axis=1)
        t = hs * ar  # 가중치를 곱해요
        c = np.sum(t, axis=1)  # 만들어진 벡터들을 더하여 맥락벡터 만듬, 근데 전체 더하는게 전체 맥락에 도움을 주나? 각 벡터를 따로 쓰는 방법 있으면 좋을듯

        self.cache = (hs, ar)
        return c

    def backward(self, dc):
        """
        역전파
        sum -> 곱셈 -> repeat 순으로 역전파
        :param dc: 
        :return: 
        """
        hs, ar = self.cache
        num_sample, time_stap, hidden_size = hs.shape
        dt = dc.reshape(num_sample, 1, hidden_size).repeat(time_stap, axis=1)  # sum노드의 역전파는 repeat노드~
        # t가 hs * ar이니까 각각 반대로 곱해주자 -> 곱셈노드는 반대로 곱해주기~
        dar = dt * hs
        dhs = dt * ar
        da = np.sum(dar, axis=2)  # repeat노드의 역전파는 sum노드~

        return dhs, da

In [6]:
# 샘플 고려 안하고 한개로 가정
time_stap = 5
hidden_size = 4

hs = np.random.randn(time_stap, hidden_size)
hs

a = np.array([0.8, 0.1, 0.03, 0.05, 0.02])
a

ar = a.reshape(time_stap, 1).repeat(hidden_size, axis=1)
ar

t = hs * ar
t

c = np.sum(t, axis=0)
c

array([-0.09466044,  0.71296525,  1.22616427, -0.65008742])

## Attention 클래스

In [7]:
# 해당 클래스는 한 타임만 처리하는 클래스임을 잊지 말자
# 이후 TimeAttention 클래스에서 모든 타임에 대해 처리해줌
class Attention:
    def __init__(self):
        self.params, self.grads = [], []
        self.attention_weight_layer = AttentionWeight()  # 어텐션 웨이트를 구해주는 친구
        self.weight_sum_layer = WeightSum()  # 구해진 어텐션 웨이트랑 웨이트섬 하는 친구
        self.attention_weight = None  # 시각화를 위해 어텐션 웨이트 저장☆

    def forward(self, hs, h):
        a = self.attention_weight_layer.forward(hs, h)  # 인코더의 hs와 현재 타임의 디코더 h값으로 어텐션 가중치 획득
        out = self.weight_sum_layer.forward(hs, a)  # 다시 인코더의 hs와 어텐션 가중치로 웨이트섬~
        self.attention_weight = a
        return out

    def backward(self, dout):
        # 당연히 반대로 weight_sum -> attention_weight
        # hs의 경우, 두 연산에 모두 사용되서 두 연산의 오차를 받아 더한 값을 최종 dhs로 사용
        dhs0, da = self.weight_sum_layer.backward(dout)
        dhs1, dh = self.attention_weight_layer.backward(da)
        dhs = dhs0 + dhs1
        return dhs, dh

## Attention 클래스

In [8]:
# 전체 타임에 대해서 처리해주는 어텐션 클래스
class TimeAttention:
    def __init__(self):
        self.params, self.grads = [], []
        self.layers = None
        self.attention_weights = None

    def forward(self, hs_enc, hs_dec):
        """
        타임별 어텐션 처리 해줌
        :param hs_enc: 인코더의 모든 히든 스테이트
        :param hs_dec: 디코더의 모든 히든 스테이트
        :return: 
        """
        num_sample, time_stap, hidden_size = hs_dec.shape
        out = np.empty_like(hs_dec)
        self.layers = []
        self.attention_weights = []

        for t in range(time_stap):  # 가지고 있는 타임 스탭만큼 반복
            layer = Attention()  # 위에서 본 어텐션을 해당 타임을 처리하기 위해 하나 만듬
            out[:, t, :] = layer.forward(hs_enc, hs_dec[:,t,:])  # 인코더의 모든 히든 스테이트와 해당 타임의 디코더의 히든 스테이트를 넘겨 어텐션 결과 저장
            self.layers.append(layer)  # 레이어 추가
            self.attention_weights.append(layer.attention_weight)  # 시각화에 쓰려고 저장

        return out

    def backward(self, dout):
        num_sample, time_stap, hidden_size = dout.shape
        dhs_enc = 0
        dhs_dec = np.empty_like(dout)

        for t in range(time_stap):  # 타임 스탭만큼 반복, 어텐션 계층은 hs와 lstm의 결과로 연산이 이루어져서 순서대로 역전파해도 각 어텐션층이 독립적?임
            layer = self.layers[t]
            dhs, dh = layer.backward(dout[:, t, :])  # 각 타임별 역전파
            dhs_enc += dhs  # 인코더쪽 최종 dh는 모든 타임의 dhs의 합
            dhs_dec[:,t,:] = dh  # 디코더의 각 타임별 dh

        return dhs_enc, dhs_dec